## Import libraries

In [1]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sem_covid.services.store_registry import store_registry

## Define constants

In [2]:
EMBEDDING_COLUMN_NAME = 'document_embeddings_use'
MINIO_SEMANTIC_SIMILARITY_BUCKET = 'semantic-similarity-matrices'

## Load data

In [3]:
df = store_registry.es_feature_store().get_features(features_name="ds_unified_dataset")

100% (6360 of 6360) |####################| Elapsed Time: 0:00:16 Time:  0:00:16


## Preprocess data

In [4]:
member_states = df[df['country']!='European Union'].copy()
european_union = df[df['country']=='European Union'].copy()
eu_timeline_european_union = european_union[european_union['doc_source']=='ds_eu_timeline'].copy()
eu_cellar_european_union = european_union[european_union['doc_source']=='ds_eu_cellar'].copy()

## Define helper functions

In [5]:
def create_similarity_matrix(x_df: pd.DataFrame, y_df: pd.DataFrame)->pd.DataFrame:
    tmp_sim_matrix = cosine_similarity(x_df[EMBEDDING_COLUMN_NAME].values.tolist(),
                                                       y_df[EMBEDDING_COLUMN_NAME].values.tolist())
    return pd.DataFrame(tmp_sim_matrix, index = x_df.index.values , columns = y_df.index.values)

## Compute semantic similarity

In [6]:
ms_x_timeline_sem_sim = create_similarity_matrix(x_df=member_states, y_df=eu_timeline_european_union)
ms_x_cellar_sem_sim = create_similarity_matrix(x_df=member_states, y_df=eu_cellar_european_union)

## Store results

In [7]:
minio_store = store_registry.minio_object_store(minio_bucket=MINIO_SEMANTIC_SIMILARITY_BUCKET)
minio_store.put_object(object_name="member_states.json",content=member_states.to_json(orient="index"))
minio_store.put_object(object_name="eu_timeline_european_union.json",content=eu_timeline_european_union.to_json(orient="index"))
minio_store.put_object(object_name="eu_cellar_european_union.json",content=eu_cellar_european_union.to_json(orient="index"))
minio_store.put_object(object_name="ms_x_timeline_sem_sim.json",content=ms_x_timeline_sem_sim.to_json(orient="index"))
minio_store.put_object(object_name="ms_x_cellar_sem_sim.json",content=ms_x_cellar_sem_sim.to_json(orient="index"))

184558323